In [ ]:
import torch
import numpy as np
import pandas as pd
import time  # Import time module
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# 🔹 1. Load dataset
train_df = pd.read_csv("ISEAR_train.csv")
test_df = pd.read_csv("ISEAR_test.csv")

# 🔹 2. Map emotions to numerical labels
unique_emotions = sorted(train_df["emotion"].unique())
emotion2label = {emotion: idx for idx, emotion in enumerate(unique_emotions)}
label2emotion = {idx: emotion for emotion, idx in emotion2label.items()}

train_df["label"] = train_df["emotion"].map(emotion2label)
test_df["label"] = test_df["emotion"].map(emotion2label)

# 🔹 3. Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df[["text", "label"]])
test_dataset = Dataset.from_pandas(test_df[["text", "label"]])

# 🔹 4. Tokenization
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# 🔹 5. Load model
num_labels = len(emotion2label)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 🔹 6. Training parameters
training_args = TrainingArguments(
    output_dir="./distilbert_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# 🔹 7. Evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

# 🔹 8. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# 🔹 9. Training with time tracking
print("⏳ Starting DistilBERT fine-tuning on ISEAR dataset...")
start_time = time.time()  # Start timing

trainer.train()

end_time = time.time()  # End timing
training_time = end_time - start_time

print(f"✅ Training completed! 🕒 Training time: {training_time / 60:.2f} minutes")

# 🔹 10. Save model
model.save_pretrained("./distilbert_finetuned")
tokenizer.save_pretrained("./distilbert_finetuned")

# 🔹 11. Evaluation on test dataset
results = trainer.evaluate()
print("📊 DistilBERT Evaluation Results:")
print(results)


Map:   0%|          | 0/6124 [00:00<?, ? examples/s]

Map:   0%|          | 0/1532 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


⏳ Starting DistilBERT fine-tuning on ISEAR dataset...


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.046100,0.964145,0.667102,0.667757,0.667102,0.665076
2,0.698100,0.896063,0.679504,0.688314,0.679504,0.676591
3,0.593400,0.927258,0.691906,0.703002,0.691906,0.693932
4,0.427900,0.997891,0.689948,0.694797,0.689948,0.690104
5,0.355000,1.119210,0.682768,0.692831,0.682768,0.685843
6,0.130200,1.225495,0.687990,0.696029,0.687990,0.690495
7,0.075800,1.338250,0.695822,0.698323,0.695822,0.696890
8,0.116900,1.468723,0.690601,0.692178,0.690601,0.691189
9,0.046300,1.509627,0.699739,0.706248,0.699739,0.702085
10,0.031900,1.528388,0.695822,0.699403,0.695822,0.697245


✅ Training completed! 🕒 Training time: 3.91 minutes


📊 DistilBERT Evaluation Results:
{'eval_loss': 1.5283881425857544, 'eval_accuracy': 0.695822454308094, 'eval_precision': 0.6994029369584668, 'eval_recall': 0.695822454308094, 'eval_f1': 0.6972453390895582, 'eval_runtime': 1.3956, 'eval_samples_per_second': 1097.716, 'eval_steps_per_second': 68.786, 'epoch': 10.0}
